In [ ]:
import nltk
import numpy as np
import re
import string
from nltk.corpus import sentence_polarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

nltk.download('sentence_polarity')
nltk.download('stopwords')

def get_data():
    pos_sents = [" ".join(sent) for sent in sentence_polarity.sents(categories='pos')]
    neg_sents = [" ".join(sent) for sent in sentence_polarity.sents(categories='neg')]

    # Розділення на тренувальну (80%) та тестову (20%) вибірки
    # Дані у sentence_polarity вже збалансовані (5331 поз, 5331 нег)

    split_pos = int(len(pos_sents) * 0.8)
    split_neg = int(len(neg_sents) * 0.8)

    train_pos = pos_sents[:split_pos]
    test_pos = pos_sents[split_pos:]

    train_neg = neg_sents[:split_neg]
    test_neg = neg_sents[split_neg:]

    train_x = train_pos + train_neg
    test_x = test_pos + test_neg

    # Створення міток (1 - позитив, 0 - негатив)
    train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
    test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = get_data()
print(f"Кількість тренувальних прикладів: {len(train_x)}")
print(f"Кількість тестових прикладів: {len(test_x)}")

[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/sentence_polarity.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Кількість тренувальних прикладів: 8528
Кількість тестових прикладів: 2134


In [ ]:
def process_text(text):
    """
    Функція для токенізації, стемінгу та видалення стоп-слів.
    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # Видаляємо посилання, хештеги тощо (стандартна очистка)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    text = re.sub(r'#', '', text)

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(text)

    clean_tokens = []
    for word in tokens:
        if (word not in stopwords_english and word not in string.punctuation):
            stem_word = stemmer.stem(word)
            clean_tokens.append(stem_word)

    return clean_tokens

def build_freqs(texts, ys):
    """
    Створення словника частот: {(слово, мітка): кількість}
    """
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, text in zip(yslist, texts):
        for word in process_text(text):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs

# Будуємо словник частот на основі тренувальних даних
freqs = build_freqs(train_x, train_y)
print(f"Розмір словника частот: {len(freqs)}")

Розмір словника частот: 18449


In [ ]:
def sigmoid(z):
    # Використовуємо clip для уникнення overflow помилок при великих від'ємних z
    z = np.clip(z, -500, 500)
    h = 1 / (1 + np.exp(-z))
    return h

def gradientDescent(x, y, theta, alpha, num_iters):
    m = x.shape[0]

    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)

        # Градієнтний крок
        # theta = theta - (alpha / m) * X^T * (h - y)
        theta = theta - (alpha / m) * np.dot(x.T, (h - y))

        # Розрахунок втрат (для моніторингу)
        if i % 100 == 0:
            epsilon = 1e-15
            h_safe = np.clip(h, epsilon, 1 - epsilon)
            J = (-1/m) * np.sum(y * np.log(h_safe) + (1-y) * np.log(1 - h_safe))
            # print(f"Ітерація {i}, Втрати: {J}") # Можна розкоментувати для налагодження

    J = (-1/m) * np.sum(y * np.log(h_safe) + (1-y) * np.log(1 - h_safe))
    return J, theta

def extract_features(text, freqs):
    word_l = process_text(text)
    x = np.zeros((1, 3))
    x[0,0] = 1 # Bias unit

    for word in word_l:
        # Підрахунок позитивних частот
        x[0,1] += freqs.get((word, 1.0), 0)
        # Підрахунок негативних частот
        x[0,2] += freqs.get((word, 0.0), 0)

    return x

#НАВЧАННЯ МОДЕЛІ

X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# 2. Ініціалізація параметрів
Y = train_y
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), alpha=1e-9, num_iters=10000)

print(f"Помилка після навчання: {J:.8f}")
print(f"Вектор ваг (theta): {theta.reshape(-1)}")

Помилка після навчання: 0.68908720
Вектор ваг (theta): [ 2.40108218e-09  1.42376909e-04 -1.43950665e-04]
